# Helper Methods

First, a method to check for emptiness among dataframes.

Then, feature engineering methods.

In [ ]:
import pandas as pd
import numpy as np

def analyze_dataframe(df):
    null_entries = {}
    inf_entries = {}

    for col in df.columns:
        null_count = df[col].isnull().sum()
        if null_count > 0:
            null_entries[col] = null_count

        inf_count = ((df[col] == np.inf) | (df[col] == -np.inf)).sum()
        if inf_count > 0:
            inf_entries[col] = inf_count

    # Find the most frequent columns for null and inf values
    most_frequent_null_column = max(null_entries, key=null_entries.get) if null_entries else None
    most_frequent_inf_column = max(inf_entries, key=inf_entries.get) if inf_entries else None

    print(100*'-')
    print("Shape:", df.shape)
    print("Null entries:", null_entries)
    print("Infinite entries:", inf_entries)
    print("Most frequent column with null entries:", most_frequent_null_column)
    print("Most frequent column with infinite entries:", most_frequent_inf_column)
    print(100*'-')

In [ ]:
def add_neighboring_wells(df):
  pad_child_count = dict()
  pad_ids = df["pad_id"].values
  for id in pad_ids:
      if id in pad_child_count:
        pad_child_count[id] += 1
      else:
        pad_child_count[id] = 1

  df["num_neighboring_wells"] = df["pad_id"].map(pad_child_count)
  return df

def euclid_surface_bh_dist(df):
    df['surface_bottom_dist'] = ((df['surface_x'] - df['bh_x'])**2 + (df['surface_y'] - df['bh_y'])**2)**0.5
    return df

def euclid_toe_dist(df):
    df['toe_dist'] = ((df['horizontal_midpoint_x'] - df['horizontal_toe_x'])**2 + (df['horizontal_midpoint_y'] - df['horizontal_toe_y'])**2)**0.5
    return df

def surface_bottom_angle(df):
   df['surface_bottom_angle'] = np.arctan2(df['surface_y'] - df['bh_y'], df['surface_x'] - df['bh_x'])
   return df

def toe_angle(df):
   df['toe_angle'] = np.arctan2(df['horizontal_midpoint_y'] - df['horizontal_toe_y'], df['horizontal_midpoint_x'] - df['horizontal_toe_x'])
   return df

# Data Cleaning Pipeline:

Drop the 4 columns that are very empty – the 4 ratios

Deal with np.inf entries

Populate null categorical thingies with ‘Unknown’

Impute null continuous thingies with average

Engineer new features based on developed pipeline

Drop features that are not necessary: standardized_operator_name, pad_id

Now, everything should be full except the ‘PeakOilRate’. Since there’s nothing to be concluded with a missing PeakOilRate, we remove those entries

Finally, one hot encode

Final number of entries should be about 19k

In [ ]:
df = pd.read_csv("/content/training (1).csv")

analyze_dataframe(df)

In [ ]:
# Drop the columns that are very empty

columns_to_drop = ['Unnamed: 0', 'ffs_frac_type', 'average_stage_length',
       'average_proppant_per_stage', 'average_frac_fluid_per_stage',
       'frac_seasoning']

df = df.drop(columns=columns_to_drop)

analyze_dataframe(df)

In [ ]:
# Deal with np.inf entries

max_finite_value = df[df['frac_fluid_to_proppant_ratio'] != np.inf]['frac_fluid_to_proppant_ratio'].max()
df['frac_fluid_to_proppant_ratio'] = df['frac_fluid_to_proppant_ratio'].replace([np.inf], max_finite_value)

analyze_dataframe(df)

In [ ]:
# Populate null categorical columns w/ unknown
categorical_cols = ['standardized_operator_name', 'relative_well_position', 'number_of_stages',
                    'batch_frac_classification', 'well_family_relationship', 'frac_type', 'pad_id']

cols_to_fill_with_minus1 = ['standardized_operator_name', 'pad_id', 'number_of_stages']

cols_to_fill_with_unknown = [col for col in categorical_cols if col not in cols_to_fill_with_minus1]

df[cols_to_fill_with_minus1] = df[cols_to_fill_with_minus1].fillna(-1)

df[cols_to_fill_with_unknown] = df[cols_to_fill_with_unknown].fillna('Unknown')

analyze_dataframe(df)

In [ ]:
# Impute null continuous features with average.

continuous_cols = [col for col in df.columns if col not in categorical_cols]

continuous_cols.remove('OilPeakRate')

print(continuous_cols)

df[continuous_cols] = df[continuous_cols].fillna(df[continuous_cols].median())

analyze_dataframe(df)

In [ ]:
# Engineer new features based on developed pipeline

add_neighboring_wells(df)
euclid_surface_bh_dist(df)
euclid_toe_dist(df)
surface_bottom_angle(df)
toe_angle(df)

analyze_dataframe(df)

In [ ]:
# Drop features that are not necessary: standardized_operator_name, pad_id

uninterpretable_cols = ['standardized_operator_name', 'pad_id']
df = df.drop(columns=uninterpretable_cols)

analyze_dataframe(df)

In [ ]:
# Remove entries with 0 OilPeakRate

df = df.dropna()

analyze_dataframe(df)

In [ ]:
# One hot encode and save

categorical_cols = ['relative_well_position', 'number_of_stages', 'batch_frac_classification',
                    'well_family_relationship', 'frac_type']

df = pd.get_dummies(df, columns = categorical_cols)

analyze_dataframe(df)

df.to_csv("/content/final_product.csv", index=True)